In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install lancedb vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.1/27.1 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.4/30.4 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7

In [ ]:
# Block 0: Mount Google Drive
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Block 1: Imports
import transformers
import re
from transformers import AutoConfig, AutoTokenizer, AutoModel, AutoModelForCausalLM
from huggingface_hub import snapshot_download
from vllm import LLM, SamplingParams
import torch
import json
import os
import shutil
import requests
import lancedb
import pandas as pd

In [ ]:
model_name = "PleIAs/RAG-1B"
local_model_path = "/content/drive/MyDrive/RAG-1B"

if not os.path.exists(local_model_path):
    print(f"Downloading {model_name} to {local_model_path}...")
    snapshot_download(repo_id=model_name, local_dir=local_model_path,
                      ignore_patterns=["*.msgpack", "*.h5", "*.ot", "*.feather"])
    print("Download complete!")
else:
    print(f"Model already exists at {local_model_path}")


Model already exists at /content/drive/MyDrive/RAG-1B


In [ ]:
# Database paths
db_path = "/content/drive/MyDrive/rag_irene/lancedb_data"
table_name = "test"


In [ ]:
# Model parameters
temperature = 0.7
max_new_tokens = 3000
top_p = 0.95
repetition_penalty = 1.2

In [ ]:
# Initialize vLLM
llm = LLM(
    model=local_model_path,
    max_model_len=8192,
    dtype="float16",  # Explicitly set float16 for T4 GPU compatibility
    gpu_memory_utilization=0.8  # Added to help with memory management
)

WARNING 11-05 13:02:55 config.py:1668] Casting torch.bfloat16 to torch.float16.
INFO 11-05 13:03:06 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='/content/drive/MyDrive/RAG-1B', speculative_config=None, tokenizer='/content/drive/MyDrive/RAG-1B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=/content/drive/MyDrive/RAG-1B

/usr/local/lib/python3.10/dist-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/usr/local/lib/python3.10/dist-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")


INFO 11-05 13:03:09 model_runner.py:1056] Starting to load model /content/drive/MyDrive/RAG-1B...
INFO 11-05 13:03:09 selector.py:224] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 11-05 13:03:09 selector.py:115] Using XFormers backend.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 11-05 13:03:49 model_runner.py:1067] Loading model weights took 2.3185 GB
INFO 11-05 13:03:51 gpu_executor.py:122] # GPU blocks: 15123, # CPU blocks: 8192
INFO 11-05 13:03:51 gpu_executor.py:126] Maximum concurrency for 8192 tokens per request: 29.54x
INFO 11-05 13:03:55 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 11-05 13:03:55 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 11-05 13:04:23 model_runner.py:1523] Graph capturing finished in 29 secs.


In [ ]:
# Connect to the LanceDB database
db = lancedb.connect(db_path)
table = db.open_table(table_name)

In [ ]:
# Block 4: Search Function
def hybrid_search(text):
    results = table.search(text, query_type="hybrid").limit(4).to_pandas()
    document = []

    for _, row in results.iterrows():
        hash_id = str(row['hash'])
        title = row['section']
        content = row['text']
        document.append(f"**{hash_id}**\n{title}\n{content}")

    return "\n\n".join(document)

In [ ]:
# Block 5: Reference Formatting Function
def format_references(text):
    ref_start_marker = '<ref text="'
    ref_end_marker = '</ref>'
    parts = []
    current_pos = 0
    ref_number = 1

    while True:
        start_pos = text.find(ref_start_marker, current_pos)
        if start_pos == -1:
            parts.append(text[current_pos:])
            break

        parts.append(text[current_pos:start_pos])
        end_pos = text.find('">', start_pos)
        if end_pos == -1:
            break

        ref_text = text[start_pos + len(ref_start_marker):end_pos].replace('\n', ' ').strip()
        ref_text_encoded = ref_text.replace("&", "&amp;").replace("<", "&lt;").replace(">", "&gt;")
        ref_end_pos = text.find(ref_end_marker, end_pos)

        if ref_end_pos == -1:
            break

        ref_id = text[end_pos + 2:ref_end_pos].strip()
        tooltip_html = f'<span class="tooltip" data-refid="{ref_id}" data-text="{ref_id}: {ref_text_encoded}"><a href="#{ref_id}">[{ref_number}]</a></span>'

        parts.append(tooltip_html)
        current_pos = ref_end_pos + len(ref_end_marker)
        ref_number += 1

    return ''.join(parts)

In [ ]:
# Block 6: Main Prediction Function
def predict(user_message):
    # Get relevant documents
    sources = hybrid_search(user_message)

    # Setup sampling parameters
    sampling_params = SamplingParams(
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_new_tokens,
        presence_penalty=repetition_penalty,
        stop=["#END#"]
    )

    # Create prompt
    prompt = f"""### Query ###\n{user_message}\n\n### Source ###\n{sources}\n\n### Analysis ###\n"""

    # Generate response
    outputs = llm.generate([prompt], sampling_params, use_tqdm=False)
    generated_text = outputs[0].outputs[0].text

    # Format response with references
    formatted_response = format_references(generated_text)

    return {
        "query": user_message,
        "sources": sources,
        "response": formatted_response
    }

In [ ]:
!pip install tantivy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 40.7 MB/s eta 0:00:00


In [ ]:
# Block 7: Usage

query = "What are the main types of risks associated with Large Language Models (LLMs)?"
result = predict(query)

# Print results
print("\nQuery:", result["query"])
print("\nSources:", result["sources"])
print("\nAnalysis:", result["response"])



Query: What are the main types of risks associated with Large Language Models (LLMs)?

Sources: **4e2d3c7a186d08a4**
I. INTRODUCTION
Large language models (LLMs) [1]–[5] that own mas-
sive model parameters pre-trained on extensive corpora, have
catalyzed a revolution in the fields of Natural Language
Processing (NLP). The scale-up of model parameters and
the expansion of pre-training corpora have endowed LLMs
with remarkable capabilities across various tasks, including
text generation [2], [4], [5], coding [2], [6], and knowledge
reasoning [7]–[10]. Furthermore, alignment techniques (e.g.,
supervised fine-tuning and reinforcement learning from human
feedback [4], [11]) are proposed to encourage LLMs to align
their behaviors with human preferences, thereby enhancing the
usability of LLMs. In practice, advanced LLM systems like
ChatGPT [12] have consistently garnered a global user base,
establishing themselves as competitive solutions for complex
NLP tasks. i
To mitigate the risks of LL

In [1]:
from RAG import retrieve
import sqlite3
import sqlite_vec
from typing import List
import struct
from connect_db import ConnectDB

connection = ConnectDB().connection
embedded_query = [0.1, 0.0, 0.1, 0.4]
query = "transformer"
documents = [1, 2, 3]
results = retrieve(connection, embedded_query, query, documents)
print(len(results))
print(results[0].keys())
print(results[1].keys())

2
dict_keys(['chunk_id', 'document_id', 'title', 'author', 'creation_date', 'text', 'vec_rank', 'fts_rank', 'combined_rank', 'vec_distance', 'fts_score'])
dict_keys(['chunk_id', 'document_id', 'title', 'author', 'creation_date', 'text', 'vec_rank', 'fts_rank', 'combined_rank', 'vec_distance', 'fts_score'])


In [63]:
results['title'] results['author']  results['creation_date']

[{'chunk_id': 30,
  'document_id': 1,
  'title': '',
  'author': '',
  'creation_date': '2024-04-10',
  'text': 'To evaluate if the Transformer can generalize to other tasks we performed experiments on English\nconstituency parsing. This task presents specific challenges: the output is subject to strong structural\nconstraints and is significantly longer than the input. Furthermore, RNN sequence-to-sequence\nmodels have not been able to attain state-of-the-art results in small-data regimes [37]. We trained a 4-layer transformer with dmodel = 1024 on the Wall Street Journal (WSJ) portion of the\nPenn Treebank [25], about 40K training sentences. We also trained it in a semi-supervised setting,\nusing the larger high-confidence and BerkleyParser corpora from with approximately 17M sentences\n[37]. We used a vocabulary of 16K tokens for the WSJ only setting and a vocabulary of 32K tokens\nfor the semi-supervised setting. We performed only a small number of experiments to select the dropout

In [2]:
results[1]

{'chunk_id': 115,
 'document_id': 2,
 'title': 'Language Models are Unsupervised Multitask Learners',
 'author': 'Alec Radford*, Jeffrey Wu*, Rewon Child, David Luan, Dario Amodei**, Ilya Sutskever**',
 'creation_date': '2019-02-14',
 'text': '“By the time we reached the top\nof one peak, the water looked blue, with some crystals on top,” said P´erez. P´erez and his friends were astonished to see the unicorn herd. These creatures could be\nseen from the air without having to move too much to see them – they were so close they could touch\ntheir horns. While examining these bizarre creatures the scientists discovered that the creatures also\nspoke some fairly regular English. P´erez stated, “We can see, for example, that they have a common\n‘language,’ something like a dialect or dialectic.”\nDr. P´erez believes that the unicorns may have originated in Argentina, where the animals\nwere believed to be descendants of a lost race of people who lived there before the arrival of humans\nin 

In [3]:
sources = [res['text'] for res in results]
titles = [res['title'] for res in results]
titles

['', 'Language Models are Unsupervised Multitask Learners']

In [4]:
def construct_prompt(results, user_message):
    hash_ids = ['**7b3a9c2d4e8f5g1h**', '**9d4e2f7c8b5a3h1i**']
    text_chunks = [res['text'] for res in results]
    titles = [res['title'] for res in results]
    sources = ''
    for i in range(len(text_chunks)):
        source = "\n".join([hash_ids[i], titles[i], text_chunks[i]])
        sources += source

    prompt = f"""### Query ###\n{user_message}\n\n### Source ###\n{sources}\n\n### Analysis ###\n"""
    return prompt


prompt = construct_prompt(results, "What is transformer?")

In [5]:
import requests
import time
import json

In [13]:
data = {
    "n_predict": 1000,  # Predictions slider (max tokens)
    "temperature": 0.25,  # Temperature slider
    "repeat_penalty": 1.0,  # Penalize repeat sequence slider
    "repeat_last_n": 256,  # Consider N last tokens for penalize slider
    "top_k": 40,  # Top-K sampling slider
    "top_p": 0.95,  # Top-P sampling slider
    "min_p": 0.05,  # Min-P sampling slider
    "tfs_z": 1,  # Tail-free sampling parameter, reduces the impact of low-probability tokens
    "typical_p": 1,  # Controls how "typical" the sampling should be (1 means standard sampling)
    "presence_penalty": 0.2,  #  penalty for tokens that have appeared at all
    "frequency_penalty": 0.2,  # penalty based on how frequently tokens have appeared
    "mirostat": 0,  # "no Mirostat" radio option
    "mirostat_tau": 5,  # Mirostat target complexity (only if mirostat enabled)
    "mirostat_eta": 0.1,  # Mirostat learning rate (only if mirostat enabled)
    "n_probs": 0,  # Show Probabilities slider
    "min_keep": 0,  # Mkeep minimum number of candidates per sampling
    "stop": ["#END#"],
    "stream": True,
    "prompt": prompt,
    "cache_prompt": False,
    "slot_id": 0
}

headers = {
    'Accept': 'text/event-stream',
    'Content-Type': 'application/json'
}

print("Sending request...")
start_time = time.time()

response = requests.post(
    'http://127.0.0.1:8080/completion',
    json=data,
    headers=headers,
    stream=False
)
end_time = time.time()
total_time = end_time - start_time

print(f"\n-------------------")
print(f"Total time: {total_time:.2f} seconds")

Sending request...


In [27]:
response.text

'data: {"content":"The","multimodal":false,"slot_id":0,"stop":false}\n\ndata: {"content":" user","multimodal":false,"slot_id":0,"stop":false}\n\ndata: {"content":" query","multimodal":false,"slot_id":0,"stop":false}\n\ndata: {"content":" is","multimodal":false,"slot_id":0,"stop":false}\n\ndata: {"content":" in","multimodal":false,"slot_id":0,"stop":false}\n\ndata: {"content":" English","multimodal":false,"slot_id":0,"stop":false}\n\ndata: {"content":" and","multimodal":false,"slot_id":0,"stop":false}\n\ndata: {"content":" seeks","multimodal":false,"slot_id":0,"stop":false}\n\ndata: {"content":" information","multimodal":false,"slot_id":0,"stop":false}\n\ndata: {"content":" about","multimodal":false,"slot_id":0,"stop":false}\n\ndata: {"content":" transformers","multimodal":false,"slot_id":0,"stop":false}\n\ndata: {"content":".","multimodal":false,"slot_id":0,"stop":false}\n\ndata: {"content":" The","multimodal":false,"slot_id":0,"stop":false}\n\ndata: {"content":" references","multimoda

In [18]:
for line in response.iter_lines():
    print(line)


b'data: {"content":"The","multimodal":false,"slot_id":0,"stop":false}'
b''
b'data: {"content":" user","multimodal":false,"slot_id":0,"stop":false}'
b''
b'data: {"content":" query","multimodal":false,"slot_id":0,"stop":false}'
b''
b'data: {"content":" is","multimodal":false,"slot_id":0,"stop":false}'
b''
b'data: {"content":" in","multimodal":false,"slot_id":0,"stop":false}'
b''
b'data: {"content":" English","multimodal":false,"slot_id":0,"stop":false}'
b''
b'data: {"content":" and","multimodal":false,"slot_id":0,"stop":false}'
b''
b'data: {"content":" seeks","multimodal":false,"slot_id":0,"stop":false}'
b''
b'data: {"content":" information","multimodal":false,"slot_id":0,"stop":false}'
b''
b'data: {"content":" about","multimodal":false,"slot_id":0,"stop":false}'
b''
b'data: {"content":" transformers","multimodal":false,"slot_id":0,"stop":false}'
b''
b'data: {"content":".","multimodal":false,"slot_id":0,"stop":false}'
b''
b'data: {"content":" The","multimodal":false,"slot_id":0,"stop":fa

In [32]:
import requests
import time
import json


def generate_with_llamafile_api(prompt):
    data = {
        "n_predict": 1000,  # Predictions slider (max tokens)
        "temperature": 0.25,  # Temperature slider
        "repeat_penalty": 1.0,  # Penalize repeat sequence slider
        "repeat_last_n": 256,  # Consider N last tokens for penalize slider
        "top_k": 40,  # Top-K sampling slider
        "top_p": 0.95,  # Top-P sampling slider
        "min_p": 0.05,  # Min-P sampling slider
        "tfs_z": 1,  # Tail-free sampling parameter, reduces the impact of low-probability tokens
        "typical_p": 1,  # Controls how "typical" the sampling should be (1 means standard sampling)
        "presence_penalty": 0.2,  #  penalty for tokens that have appeared at all
        "frequency_penalty": 0.2,  # penalty based on how frequently tokens have appeared
        "mirostat": 0,  # "no Mirostat" radio option
        "mirostat_tau": 5,  # Mirostat target complexity (only if mirostat enabled)
        "mirostat_eta": 0.1,  # Mirostat learning rate (only if mirostat enabled)
        "n_probs": 0,  # Show Probabilities slider
        "min_keep": 0,  # Mkeep minimum number of candidates per sampling
        "stop": ["#END#"],
        "stream": True,
        "prompt": prompt,
        "cache_prompt": False,
        "slot_id": 0
    }

    headers = {
        'Accept': 'text/event-stream',
        'Content-Type': 'application/json'
    }

    try:
        print("Sending request...")
        start_time = time.time()

        response = requests.post(
            'http://127.0.0.1:8080/completion',
            json=data,
            headers=headers,
            stream=False
        )

        print("Receiving response...")
        full_response = ""
        for line in response.iter_lines():
            if line:
                decoded_line = line.decode('utf-8')
                if decoded_line.startswith('data: '):
                    text = decoded_line[6:]  # Remove 'data: ' prefix
                    if text != '[DONE]':
                        try:
                            json_response = json.loads(text)
                            if 'content' in json_response:
                                content = json_response['content']
                                full_response += content
                        except json.JSONDecodeError:
                            print("Failed to parse JSON:", text)

        end_time = time.time()
        total_time = end_time - start_time

        print(f"\n-------------------")
        print(f"Total time: {total_time:.2f} seconds")

    except Exception as e:
        print(f"Error occurred: {str(e)}")
        print(f"Error type: {type(e)}")
        
    return full_response


In [33]:
out_response = generate_with_llamafile_api(prompt)

Sending request...
Receiving response...

-------------------
Total time: 26.25 seconds


In [61]:
import re


def replace_references(text, ref_map):
    def ref_replacer(match):
        ref_name = re.findall(r'<ref name="[^"]+">', match.group(0))[0]
        replacement = ref_map[ref_name]
        return replacement

    updated_text = re.sub(r'<ref name="[^"]+">[^<]+</ref>', ref_replacer, text)
    return updated_text


def return_span(number=1):
    return f"""<span style="border: 1px solid #007bff; padding: 2px; border-radius: 3px; background-color: #e7f3ff; color: #007bff; font-size: 12px;">{number}</span>"""


def convert_input_msg_to_html(answer):
    pattern = r'###\s*Answer\s*###'
    match = re.search(pattern, answer)
    start_answer = match.end()
    end_answer = answer.find("#END#") if answer.find("###") == -1 else len(answer)
    answer = answer[start_answer:end_answer]
    references = re.findall(r'<ref name="[^"]+">', answer)
    print(references)
    ref_map = {}
    reference_count = 1
    for ref in references:
        if ref not in ref_map:
            ref_map[ref] = return_span(reference_count)
            reference_count += 1
    updated_text = replace_references(answer, ref_map)
    return updated_text


from IPython.core.display import HTML
out = convert_input_msg_to_html(out_response)          
HTML(out)

['<ref name="7b3a9c2d4e8f5g1h">', '<ref name="7b3a9c2d4e8f5g1h">', '<ref name="9d4e2f7c8b5a3h1i">', '<ref name="9d4e2f7c8b5a3h1i">']


In [68]:
out = """
The query pertains to understanding the role of digital tools in enhancing student engagement and motivation during online learning. To address this, we can draw upon several references that provide comprehensive insights into this topic.

Firstly, the article by **7b3a9c2d4e8f5g1h** discusses how digital tools can be effectively used to improve student engagement and motivation in online learning environments. It highlights that incorporating interactive elements such as quizzes, polls, and real-time feedback mechanisms can significantly enhance student engagement. The article also notes that providing personalized content and scheduling regular breaks can further boost motivation and reduce burnout among students<span style="border: 1px solid #007bff; padding: 2px; border-radius: 3px; background-color: #e7f3ff; color: #007bff; font-size: 12px;">1</span>.

Another reference, **6c7a8b9c10d11e12f**, elaborates on the importance of creating a supportive and interactive learning environment. It suggests that fostering a community among students through group discussions, virtual coffee breaks, and peer mentoring can significantly improve motivation and reduce feelings of isolation during online learning. This article also emphasizes the role of timely and effective communication in maintaining a positive learning experience<span style="border: 1px solid #007bff; padding: 2px; border-radius: 3px; background-color: #e7f3ff; color: #007bff; font-size: 12px;">2</span>.

The blog post by **4e5f6g7h8i9j** provides additional context on the practical implementation of these strategies. It shares examples of successful digital tools and practices that have been used in various online learning environments. This personal account can offer practical tips and anecdotes that can help readers visualize how these strategies might be applied in their own teaching contexts<span style="border: 1px solid #007bff; padding: 2px; border-radius: 3px; background-color: #e7f3ff; color: #007bff; font-size: 12px;">3</span>.

Lastly, the article by **2c3d4e5f6g7h** provides a broader perspective on the impact of digital tools on student motivation. It discusses how technology can both enhance and detract from student engagement, depending on how it is used. The article recommends a balanced approach where digital tools are used to supplement traditional teaching methods rather than replace them entirely. This ensures that students do not become reliant on technology alone and develop critical thinking skills that are essential for long-term success<span style="border: 1px solid #007bff; padding: 2px; border-radius: 3px; background-color: #e7f3ff; color: #007bff; font-size: 12px;">4</span>.

In summary, the use of digital tools in online learning can significantly enhance student engagement and motivation if used effectively. Strategies such as incorporating interactive elements, fostering a supportive community, and maintaining effective communication are crucial for creating a positive learning environment.
"""
HTML(out)